In [72]:
# %pip install torch
# %pip install torchtext

In [73]:
# !unzip colab.zip

In [74]:
import torch
torch.manual_seed(42) # Set Seed for reproducibility
import csv
import numpy as np
import pandas as pd
from nltk import RegexpTokenizer
import re
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader, Dataset
from numpy.linalg import norm
import scipy as sp
# import counter
from collections import Counter
import math
import random

In [75]:
sizeOfTrainData = int(0.2*120000)

In [76]:
def getDescriptions(csvFilePath, numOfRows = 0):
    if numOfRows == 0:
        df = pd.read_csv(csvFilePath, usecols=['Description'])
    else:
        df = pd.read_csv(csvFilePath, nrows=numOfRows, usecols=['Description'])
    # print(df.shape)
    # df.head()
    textData = df['Description']
    return textData

# Dataset

In [77]:

UNKNOWN_TOKEN = "<UNK>"
NUMBER_TOKEN = "<NUM>"
START_TOKEN = "<START>"
END_TOKEN = "<END>"
PAD_TOKEN = "<PAD>"
class skipGramDataset(Dataset):
    def __init__(self, data : list[str],  sample: float , windowSide : int, wordVocab:Vocab|None = None):
        self.data = data
        self.wordVocab = wordVocab
        # if wordVocab is None:
        #     print("wordVocab needs to be provided")
        #     exit(1)
        self.tokenizedData = [torch.tensor(self.wordVocab.lookup_indices(self.tokenize(des)), dtype=torch.long) for des in self.data]
        self.frequencies = None
        self.sample = sample
        self.retainProbs = self.getRetainProbs()
        self.windowSide = windowSide


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # des =  self.tokenize(self.data[idx])
        # if self.wordVocab is not None:
        #     return
        return self.tokenizedData[idx]

    def tokenize(self, des:str):
        des = des.lower()
        tokDes = RegexpTokenizer(r'\w+').tokenize(des)
        # add start and end token
        tokDes = [START_TOKEN] + tokDes + [END_TOKEN]
        # replace numbers with <NUM>
        tokDes = [re.sub(r'\d+', NUMBER_TOKEN, word) for word in tokDes]
        return tokDes

    def getFrequencies(self):
        freq = Counter()
        for des in self.tokenizedData:
            freq.update(des)
        return freq

    def  getRetainProbs(self):
        self.frequencies = self.getFrequencies()
        retainProbs = {}
        for word in self.frequencies:
            freq = self.frequencies[word]
            retainProbs[word] = (math.sqrt(freq/self.sample) + 1) * (self.sample/freq)
        return retainProbs

    def collate(self, batch):
        """ create skip-gram pairs  by including subsampling of frequent words"""
        for des in batch:
            input = []
            context = []
            for i in range(len(des)):
                if random.random() > self.retainProbs[des[i]]:
                    continue
                else :
                    for j in range(max(0, i - self.windowSide), min(len(des), i + self.windowSide + 1)):
                        if i == j:
                            continue
                        if random.random() > self.retainProbs[des[j]]:
                            continue
                        else:
                            input.append(des[i])
                            context.append(des[j])
        return torch.tensor(input, dtype=torch.long), torch.tensor(context, dtype=torch.long)

# load Vocab


In [78]:
wordVocab = torch.load("trainWordVocab"+str(sizeOfTrainData)+".pth")
print("Vocab size: ", len(wordVocab))

Vocab size:  19369


# parameters

In [79]:
numNegSamples = 10
# negSamplingArrayLength = 100000
negSamplingExponent = 0.75
sample = 0.001
windowSize = 5
batchSize = 100
embeddingSize = 100
learningRate = 0.001
numOfEpochs = 10


# Negative Sampler

In [80]:
class NegativeSampler(torch.nn.Module):
    def __init__(self,  negSamplingExponent: float, wordFrequencies: dict, negSamplingArrayLength: int= 1):
        super().__init__()
        self.negSamplingArrayLength = negSamplingArrayLength
        self.negSamplingExponent = negSamplingExponent
        self.wordFrequencies = wordFrequencies
        self.negSamplingKeys = self.wordFrequencies.keys()
        # self.keysToIndex = {word: i for i, word in enumerate(self.negSamplingKeys)}
        self.negSamplingArray = self.getNegSamplingArray()

    def getNegSamplingArray(self):
        exponentFrequencies = {word: freq**self.negSamplingExponent for word, freq in self.wordFrequencies.items()}
        denominator = sum(exponentFrequencies.values())
        # negSamplingArray = []
        # for word, freq in exponentFrequencies.items():
        #     negSamplingArray = negSamplingArray + [int(word)] * (max(1, int((freq/ denominator) * self.negSamplingArrayLength)))
        # # update the negSamplingArrayLength to the actual length of the array
        # self.negSamplingArrayLength = len(negSamplingArray)
        # return negSamplingArray
        cumulativeFrequencies = np.empty(len(self.negSamplingKeys), dtype = float)
        prev = 0
        for idx, word in enumerate(self.negSamplingKeys):
          freq = exponentFrequencies[word]
          cumulativeFrequencies[idx] = prev + float(freq)/denominator
          prev = cumulativeFrequencies[idx]
        return cumulativeFrequencies

    def getNegativeSamples(self, batchSize: int, numNegSamples: int ):
        negativeSamples = []
        # for _ in range(batchSize):
        #     negativeSamples.append(random.choices(self.negSamplingArray, k=numNegSamples))
        for _ in range(batchSize):
          randNums = [random.random() for _ in range(numNegSamples)]
          samples = [int(self.negSamplingKeys[self.lower_bound(num)]) for num in randNums]
          negativeSamples.append(samples)
        return torch.as_tensor(np.array(negativeSamples))

    def lower_bound(self, target):
      l, r = 0, len(self.negSamplingArray) - 1
      while l <= r:
          mid = l + (r - l) / 2
          if self.negSamplingArray[mid] == target:
            return mid
          if self.negSamplingArray[mid] > target:
              r = mid - 1
          else:
              l = mid + 1
      return l

# Word2Vec Skip-gram model

In [81]:
class SkipGram(torch.nn.Module):
    def __init__(self, wordVocab:Vocab, embeddingDim:int):
        super().__init__()
        self.wordVocab = wordVocab
        self.targetEmbeddingModule = torch.nn.Embedding(len(wordVocab), embeddingDim)
        self.contextEmbeddingModule = torch.nn.Embedding(len(wordVocab), embeddingDim)

    def forward(self, input, context):
        targetEmbeddings = self.targetEmbeddingModule(input)
        numTargets = targetEmbeddings.shape[0]
        embedDIm = targetEmbeddings.shape[1]
        targetEmbeddings = targetEmbeddings.view(numTargets, 1, embedDIm)
        contextEmbeddings = self.contextEmbeddingModule(context)
        contextEmbeddings = contextEmbeddings.permute(0, 2, 1)

        # Calculate the dot products
        dotProducts = targetEmbeddings.bmm(contextEmbeddings)
        dotProducts = dotProducts.view(dotProducts.shape[0], dotProducts.shape[2])
        return dotProducts

    def getEmbeddings(self):
        return self.targetEmbeddingModule.weight.data

    def normalizeEmbeddings(self):
        # Normalize the embeddings in targetEmbeddingModule
        embeddings = self.targetEmbeddingModule.weight.data # returns a numpy array
        norms = (embeddings**2).sum(1)**0.5
        norms = norms.reshape(norms.shape[0], 1)
        # normalize the embeddings
        return embeddings/norms


    def getSimilarity(self, word1:str, word2:str):
        word1Index = self.wordVocab.get_stoi(word1)
        word2Index = self.wordVocab.get_stoi(word2)
        embeddedNorms = self.normalizeEmbeddings()
        word1Embedding = embeddedNorms[word1Index]
        word2Embedding = embeddedNorms[word2Index]
        return torch.nn.functional.cosine_similarity(word1Embedding, word2Embedding).item()

# train dataset


In [82]:
trainData = getDescriptions('./data/train.csv', sizeOfTrainData)
print(trainData[:5])
trainDataset = skipGramDataset(trainData, sample , windowSize, wordVocab)
print(trainDataset[6])
trainWordFrequencies = trainDataset.frequencies
trainDataloader = DataLoader(trainDataset, batch_size=batchSize, shuffle=True, collate_fn= trainDataset.collate)

0    London - The British Broadcasting Corporation,...
1    Embattled insurance broker #39;s banks agree t...
2    AP - Derek Jeter turned a season that started ...
3    When the Genesis capsule comes back to Earth w...
4     NEW YORK (Reuters) - U.S. stocks were set to ...
Name: Description, dtype: object
tensor([   2,   76,  101, 1888,   61,   16,  127,   50, 3777, 1426, 2432, 3842,
         101,   13,   18,    0,  512,  171,    9,   25,  732,    7, 2062,    5,
         862, 6746,   21,   20,  642,    7, 3173,   50,  833,    3])


# val dataset

In [83]:
valData = getDescriptions('./data/val.csv')
valDataset = skipGramDataset(valData, sample, windowSize, wordVocab)
valDataloader = DataLoader(valDataset, batch_size=batchSize, shuffle=False, collate_fn= valDataset.collate)

# Train

In [84]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [85]:
# negative sampler for training data
trainNegativeSampler = NegativeSampler(negSamplingExponent, trainWordFrequencies).to(device)


In [86]:
mySkipGram = SkipGram(wordVocab, embeddingSize)
lossFn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(mySkipGram.parameters(), lr=learningRate)

# send the model to the device
mySkipGram = mySkipGram.to(device)

# Train the model
for epochNum in range(numOfEpochs):
    mySkipGram.train()
    for batchNum, (inputs, contexts) in enumerate(trainDataloader):
        # Zero the gradients
        optimizer.zero_grad()

        # Move the data to the device
        inputs = inputs.to(device)
        posLabels = contexts.to(device)
        negLabels = trainNegativeSampler.getNegativeSamples(posLabels.shape[0], numNegSamples).to(device)
        contexts = torch.cat([posLabels.view(posLabels.shape[0],1), negLabels], dim=1)

        # targets tensor
        yPos = torch.ones(posLabels.shape[0], 1).to(device)
        yNeg = torch.zeros(negLabels.shape[0], numNegSamples).to(device)
        targets = torch.cat([yPos, yNeg], dim=1)

        # Forward pass
        outputs = mySkipGram(inputs , contexts)
        loss = lossFn(outputs, targets)

        # Backward pass and update
        loss.backward()
        optimizer.step()

        if batchNum % 100 == 0:
            print('Epoch: ', epochNum, ', Batch: ', batchNum, ', Loss: ', loss.item())

    # Evaluate the model on the validation set
    mySkipGram.eval()
    numCorrect = 0
    numTotal = 0
    evalLoss = 0
    # turn off gradients
    with torch.no_grad():
        for batchNum , (inputs, contexts) in enumerate(valDataloader):
            # Move the data to the device
            inputs = inputs.to(device)
            posLabels = contexts.to(device)
            negLabels = trainNegativeSampler.getNegativeSamples(posLabels.shape[0], numNegSamples).to(device)
            contexts = torch.cat([posLabels.view(posLabels.shape[0],1), negLabels], dim=1)

            # targets tensor
            yPos = torch.ones(posLabels.shape[0], 1).to(device)
            yNeg = torch.zeros(negLabels.shape[0], numNegSamples).to(device)
            targets = torch.cat([yPos, yNeg], dim=1)

            # Forward pass
            outputs = mySkipGram(inputs, contexts)
            loss = lossFn(outputs, targets)

            evalLoss += loss.item()
            # Get the predictions
            predictions = torch.sigmoid(outputs)
            predictions = torch.argmax(predictions, dim=1)
            targets = torch.argmax(targets, dim=1)
            numCorrect += torch.sum(predictions == targets).item()
            numTotal += targets.shape[0]
    print('Epoch: ', epochNum, ', Validation Accuracy: ', numCorrect / numTotal, ', Validation Loss: ', evalLoss / numTotal)

# Save the model
modelPath = f"skipGram_{embeddingSize}_{numNegSamples}_{negSamplingExponent}_{learningRate}_{numOfEpochs}.pth"
torch.save(mySkipGram.getEmbeddings(), modelPath)

KeyError: tensor(2)

# Test

In [ ]:
testWords = []
for word in testWords:
    # get similar words
    pass


In [ ]:
# %pip install torch
# %pip install torchtext

In [ ]:
# %pip install torch
# %pip install torchtext

In [ ]:
# !unzip colab.zip

In [ ]:
# !unzip colab.zip